In [ ]:
import pysster

In [ ]:
from pysster import Data, Model
from pysster import utils

In [ ]:
pos = 'example-input/Mukherjee-PAR-CLIP/QKI_HEK293_PARCLIP/fold-0/positive.fold-0.fasta'
neg = 'example-input/Mukherjee-PAR-CLIP/QKI_HEK293_PARCLIP/fold-0/negative-1.fold-0.fasta'

In [ ]:
pysster_data = Data.Data([pos, neg], alphabet='ACGT')


# with open(str(out_path.joinpath('data_summary.txt')), 'w') as f:
#     print(pysster_data.get_summary(), file=f)


# In[ ]:
pysster_model = Model.Model({}, pysster_data, seed=42)

In [ ]:
pysster_model.model.save('model.pkl.h5')

In [1]:
import tensorflow as tf

model = tf.keras.models.load_model('models/Mukherjee-PAR-CLIP/QKI_HEK293_PARCLIP/fold-0/negative-1/model.pkl', compile=False)
model.summary()

2023-01-16 13:39:26.350895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-01-16 13:41:25.757430: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-16 13:41:25.784846: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-16 13:41:25.831377: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2f:00.0 name: Quadro P6000 computeCapability: 6.1
coreClock: 1.645GHz coreCount: 30 deviceMemorySize: 23.88GiB deviceMemoryBandwidth: 403.49GiB/s
2023-01-16 13:41:25.831432: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-01-16 13:41:25.842801: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-01-

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 4)]          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 4)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 383, 150)          10950     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 191, 150)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 191, 150)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 174, 150)          405150    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 87, 150)           0   

In [2]:
pred = model(tf.random.uniform(shape=(1, 400, 4)))
pred.shape

2023-01-16 13:41:46.446028: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2023-01-16 13:41:54.818750: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-01-16 13:41:57.630791: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


TensorShape([1, 2])

In [4]:


base2int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def sequence2int(sequence, mapping=base2int):
    return [mapping.get(base, 999) for base in sequence]

def sequence2onehot(sequence, mapping=base2int):
    return tf.one_hot(sequence2int(sequence, mapping), depth=4)

def load_fasta(fasta):
    with open(fasta) as f:
        for line in f:
            assert line[0] == '>'
            header, sequence = line.strip(), f.readline().strip()
            name, *_ = header[1:].split(':')
            yield tf.cast(sequence2onehot(sequence), tf.float32)

def load_dataset(fasta):
    dataset = tf.data.Dataset.from_generator(lambda: load_fasta(fasta), output_types=tf.float32)
    dataset = dataset.batch(128)
    return dataset


In [11]:
dataset = load_dataset('inputs/Mukherjee-PAR-CLIP/QKI_HEK293_PARCLIP/fold-0/positive.fold-0.UPPER.fasta')
dataset.element_spec

TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)

In [12]:
for batch in dataset.take(2):
    pred_batch = model(batch)[:, 0]
    for score in pred_batch:
        print(score)

tf.Tensor(0.9990243, shape=(), dtype=float32)
tf.Tensor(0.99920017, shape=(), dtype=float32)
tf.Tensor(0.97769743, shape=(), dtype=float32)
tf.Tensor(0.9993291, shape=(), dtype=float32)
tf.Tensor(0.9998776, shape=(), dtype=float32)
tf.Tensor(0.9998739, shape=(), dtype=float32)
tf.Tensor(0.99961084, shape=(), dtype=float32)
tf.Tensor(0.9975311, shape=(), dtype=float32)
tf.Tensor(0.99993086, shape=(), dtype=float32)
tf.Tensor(0.9991572, shape=(), dtype=float32)
tf.Tensor(0.99999845, shape=(), dtype=float32)
tf.Tensor(0.9996662, shape=(), dtype=float32)
tf.Tensor(0.999974, shape=(), dtype=float32)
tf.Tensor(0.99997246, shape=(), dtype=float32)
tf.Tensor(0.99971265, shape=(), dtype=float32)
tf.Tensor(0.99979705, shape=(), dtype=float32)
tf.Tensor(0.9999199, shape=(), dtype=float32)
tf.Tensor(0.9999324, shape=(), dtype=float32)
tf.Tensor(0.999851, shape=(), dtype=float32)
tf.Tensor(0.994286, shape=(), dtype=float32)
tf.Tensor(0.9997168, shape=(), dtype=float32)
tf.Tensor(0.99541426, shape=(